In [1]:
import pandas as pd
import collections as c
import numpy as np
import heapq

In [2]:
class RuleData:
    def __init__(self,rule=((),()) ,support=0.0, confidence=0.0, expFlag=True):
        self.rule=rule
        self.support=support
        self.confidence=confidence
        self.expFlag=expFlag
    def __lt__(self,r):
        return self.support<r.support
    def __str__(self):
        return str(str(self.rule), str(self.support), str(self.confidence), str(self.expFlag))

In [3]:
#data = pd.read_csv(r'Datasets/chess.txt',sep=" ",header=None)
data = pd.read_csv(r'Datasets/sample.txt',sep=" ",header=None)
data.head()
#df = pd.read_csv(filepath, chunksize=1, header=None, encoding='utf-8')

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN


In [4]:
# data.drop(37,axis=1,inplace=True)
# data.head()
# #data

In [5]:
itemset = list(set(data.values.flatten()))
itemset

[nan, 1.0, 2.0, 3.0, 4.0, 5.0, nan, nan, nan, nan, nan, nan]

In [6]:
tidsets={}
for i in range(len(data)):
    for j in range(len(data.columns)):
        val = data.iloc[i][j]
        if pd.notna(val):
            if val in tidsets:
                tidsets[val].append(i)
            else:
                tidsets[val]=[i]
tidsets

{1.0: [0, 1, 3, 4],
 2.0: [0, 1, 2, 3, 4, 5],
 3.0: [0, 2, 4, 5],
 4.0: [0, 1, 3, 5],
 5.0: [0, 1, 2, 3, 4]}

In [7]:
# tidsets=[[]]
# for i in itemset:
#     tidsets.append([])
# for i in range(len(data)):
#     for j in range(len(data.columns)):
#         val = data.iloc[i][j]
#         tidsets[val-1].append(i)
# tidsets

In [8]:
#[j for j in range(len(data)) if max((data==itemset[0]).iloc[j])==True]
data.iloc[0].values

array([1., 2., 3., 4., 5.])

In [9]:
minsup=0
minconf=0.8
k=3
l=list()
r=list()

In [10]:
# itemcount = dict(c.Counter(data.values.flatten()))
# itemcount

In [11]:
class MyHeap(object):
    def __init__(self, initial=None, key=lambda x:x):
        self.key = key
        if initial:
                self._data = [(self.key(item), item) for item in initial]
                heapq.heapify(self._data)
        else:
                self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item), item))

    def pop(self):
        return heapq.heappop(self._data)[1]
    def length(self):
        return len(self._data)
    def __str__(self):
        str(self._data)

In [12]:
def support(rule=((),())):
    if(isinstance(rule[0],int) or isinstance(rule[0],float)):
        if(isinstance(rule[1],int) or isinstance(rule[1],float)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int) or isinstance(rule[1],float)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    items=rule[0]+rule[1]
    sumRule=0
    for i in range(len(data)):
        flagRule=True
        for j in items:
            if j not in list(data.iloc[i,:]):
                flagRule=False
                break
        if(flagRule):
            sumRule+=1
    return sumRule/len(data)

In [13]:
support(((2,3),(1)))

0.3333333333333333

In [14]:
def confidence(rule=((),())):
    #print(rule)
    if(isinstance(rule[0],int) or isinstance(rule[0],float)):
        if(isinstance(rule[1],int) or isinstance(rule[1],float)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int) or isinstance(rule[1],float)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    items=rule[0]+rule[1]
    #print(rule)
    #print(items)
    sumRule=0
    sumAntecedant=0
    #The following can be optimized by bringing both under single loop block
    for i in range(len(data)):
        flagRule=True
        for j in items:
            if j not in list(data.iloc[i,:]):
                flagRule=False
                break
        if(flagRule):
            sumRule+=1
            
    for i in range(len(data)):
        flagAntecedant=True
        for j in rule[0]:
            if j not in list(data.iloc[i,:]):
                flagAntecedant=False
                break
        if(flagAntecedant):
            sumAntecedant+=1
            
    return sumRule/sumAntecedant

In [15]:
ruleDetails=dict()
L=MyHeap(key=lambda x:-x.support)

In [16]:
class Student:
    def __init__(self, age, roll):
        self.age=age
        self.roll=roll

In [17]:
J=MyHeap(key=lambda x:x.roll)

In [18]:
J.push(Student(10,"aaa"))
J.push(Student(20,"bbb"))
J.push(Student(15,"aba"))

In [19]:
J._data

[('aaa', <__main__.Student at 0x122c76ed0>),
 ('bbb', <__main__.Student at 0x122c76e90>),
 ('aba', <__main__.Student at 0x122c76f10>)]

In [20]:
J.pop().roll

'aaa'

In [21]:
confidence(((2,3),(1)))

0.5

In [22]:
def save(L,k,sup,conf,rule=((),())):
    if(isinstance(rule[0],int) or isinstance(rule[0],float)):
        if(isinstance(rule[1],int) or isinstance(rule[1],float)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int) or isinstance(rule[1],float)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    expFlag=True
    if(rule[0][0]==largestElement):
        expFlag=False
    L.push(RuleData(rule,sup,conf,expFlag))
    if(expFlag):
        R.push(RuleData(rule,sup,conf,expFlag))
    ruleData=RuleData()
    q=list()
    flag=False
    for i in range(min(L.length(),k)):
        q+=[L.pop()]
        flag=True
    if(L.length()>0):
        ruleData=L.pop()
        while(ruleData.support==q[-1].support):
            q+=[ruleData]
            if(L.length()>0):
                ruleData=L.pop()
            else:
                break
        
        
    if(not flag):
        return L
    else:
        minsup=q[-1].support
        return MyHeap(q, key=lambda x:-x.support) 

In [23]:
largestElement=max(tidsets.keys())

In [24]:
R=MyHeap(key=lambda x:-x.support)

In [25]:
items = list(tidsets.keys())
print(items)
for i in range(len(items)):
    for j in range(i+1,len(items)):
        if len(tidsets[items[i]])/len(data)>=minsup and len(tidsets[items[j]])/len(data)>=minsup:
            
            conf1=confidence(((items[i],),(items[j],)))
            conf2=confidence(((items[j],),(items[i],)))
            if conf1>=minconf:
                ruleDetails[(items[i],items[j])]=support(((items[i],),(items[j],)))
                print(conf1)
                print(((items[i],),(items[j],)))
                L = save(L,k,ruleDetails[(items[i],items[j])],conf1,((items[i],),(items[j],)))
                
            if conf2>=minconf:
                ruleDetails[(items[j],items[i])]=support(((items[j],),(items[i],)))
                print(conf2)
                print(((items[j],),(items[i],)))
                L = save(L,k,ruleDetails[(items[j],items[i])],conf2,((items[j],),(items[i],)))
                

[1.0, 2.0, 3.0, 4.0, 5.0]
1.0
((1.0,), (2.0,))
1.0
((1.0,), (5.0,))
0.8
((5.0,), (1.0,))
1.0
((3.0,), (2.0,))
1.0
((4.0,), (2.0,))
0.8333333333333334
((2.0,), (5.0,))
1.0
((5.0,), (2.0,))


In [26]:
for i in L._data:
    print(i[1].support,i[1].rule,i[1].expFlag)

0.8333333333333334 ((5.0,), (2.0,)) False
0.8333333333333334 ((2.0,), (5.0,)) True
0.6666666666666666 ((5.0,), (1.0,)) False
0.6666666666666666 ((3.0,), (2.0,)) True
0.6666666666666666 ((4.0,), (2.0,)) True
0.6666666666666666 ((1.0,), (5.0,)) True
0.6666666666666666 ((1.0,), (2.0,)) True


In [27]:
L._data

[(-0.8333333333333334, <__main__.RuleData at 0x122c76090>),
 (-0.8333333333333334, <__main__.RuleData at 0x1221b4f10>),
 (-0.6666666666666666, <__main__.RuleData at 0x122bff390>),
 (-0.6666666666666666, <__main__.RuleData at 0x121f554d0>),
 (-0.6666666666666666, <__main__.RuleData at 0x122c5f910>),
 (-0.6666666666666666, <__main__.RuleData at 0x122c5fd10>),
 (-0.6666666666666666, <__main__.RuleData at 0x122c9f790>)]

In [28]:
data

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN
5,2,3,4,NaN,NaN


In [29]:
def ExpandL(ruleData,L,R,k):
    candidateItems=set()
    rule=ruleData.rule;
    items=set(list(rule[0])+list(rule[1]))
    for i in range(len(data)):
        flag=True
        for j in items:
            if j not in data.iloc[i,:]:
                flag=False
        if(flag):
            for x in data.iloc[i,:]:
                if x not in items and x>max(rule[0]):
                    candidateItems.add(x)
    for c in candidateItems:
        rnew=(ruleData.rule[0]+(c,), ruleData.rule[1])
        sup=support(rnew)
        if  sup>=minsup:
            conf=confidence(rnew)
            if conf>=minconf:
                L=save(L,k,sup,conf,rnew)

In [30]:
def ExpandR(ruleData,L,R,k):
    candidateItems=set()
    rule=ruleData.rule;
    items=set(list(rule[0])+list(rule[1]))
    for i in range(len(data)):
        flag=True
        for j in items:
            if j not in data.iloc[i,:]:
                flag=False
        if(flag):
            for x in data.iloc[i,:]:
                if x not in items and x>max(rule[1]):
                    candidateItems.add(x)
    for c in candidateItems:
        rnew=(ruleData.rule[0], ruleData.rule[1]+(c,))
        sup=support(rnew)
        if  sup>=minsup:
            conf=confidence(rnew)
            if conf>=minconf:
                L=save(L,k,sup,conf,rnew)

In [ ]:
while(R.length()!=0):
    ruleData=R.pop()
    if ruleData.support>=minsup:
        if(ruleData.expFlag):
            L=ExpandL(ruleData,L,R,k)
            L=ExpandR(ruleData,L,R,k)
        else:
            L=ExpandR(ruleData, L, R, k)
    q=list()
    flag=False
    if R.length()>0:
        ruleData=R.pop()
        while(ruleData.support>=minsup):
            q+=[ruleData]
            flag=True
    if(flag):
        R =  MyHeap(q,lambda x:-x.support)

In [ ]:
L.length()

In [ ]:
ruleDetails

In [ ]:
# summ=0
# for i in range(data.shape[0]):
#     flag=True
#     for j in ((2,3)):
#         if j not in list(data.iloc[i,:]):
#             flag=False
#             break
#     if(flag):
#         print(i)
#         summ+=1